In [1]:
import os
import yaml
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import List, TypedDict, Annotated
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
"""
from langchain.vectorstores import FAISS
from langchain_core.graph import Graph, Node, StartNode, StopNode
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
"""

# Load environment variables
load_dotenv()
api_key = os.environ["groq"]

# Initialize LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key
)


In [2]:
llm.invoke("hi").content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [3]:
llm.invoke("where is taj mahal?").content

"The Taj Mahal is located in Agra, a city in the Indian state of Uttar Pradesh. It is situated on the southern bank of the Yamuna River, approximately 230 kilometers (143 miles) south of New Delhi, the capital city of India.\n\nThe exact address of the Taj Mahal is:\nTaj Mahal, Dharmapuri, Forest Colony, Tajganj, Agra, Uttar Pradesh 282001, India\n\nIt's one of the most famous and iconic landmarks in India, and a popular tourist destination, attracting millions of visitors each year."

In [14]:
resume,personal_info="","""Strategic and results-driven talent acquisition leader with a proven track record in recruiting and talent management across diverse industries. I specialize in designing and executing scalable recruitment strategies that align with organizational goals and elevate the candidate experience.

With a deep understanding of data-driven hiring, I leverage analytics to streamline processes, enhance team performance, and drive informed decision-making. I’m not just focused on filling roles—I build and lead high-performing recruitment teams capable of managing high-volume, cross-functional hiring with precision and efficiency.

My approach has consistently improved hiring outcomes, aligning recruitment operations with business objectives while ensuring an exceptional stakeholder experience. Backed by a strong foundation in Customer Relationship Management and Human Resources, I am passionate about delivering excellence in every aspect of talent acquisition."""
with open('/workspaces/github_analysis/data/job_description.txt', 'r') as file:
    job_description=file.read()
loader = PyPDFLoader("/workspaces/github_analysis/data/resume_with_capg.pdf")
documents = loader.load()
for i in documents:
    resume+=(i.page_content.strip())
    
with open('/workspaces/github_analysis/src/prompts.yaml', 'r') as file:
    data = yaml.safe_load(file)
    
def intro():

    print("*"*100)
    for i,j in enumerate(data.keys()):
        print(f"{i+1}. {j}")
def inputs():
    i=input("Enter the number corresponding to the key you want to use: ")
    print("selected number is ",int(i))
    key=list(data.keys())[int(i)-1]
    return key

def func(key):
    input_variables=list(data[key]["inputs"].keys())
    template=data[key]["prompt"].format(resume=resume, job_description=job_description,personal_info=personal_info)
    prompt_template = PromptTemplate(
        input_variables=input_variables,
        template=template
    )
    print(prompt_template,input_variables)
    chain=prompt_template | llm
    response=chain.invoke(data[key]["inputs"]).content
    return response

In [13]:
intro()


****************************************************************************************************
1. resume_and_job_decription_rewrite
2. cold_mail_and_person_info
3. cold_mail_no_personal_info


In [10]:
x=inputs()

selected number is  2


In [11]:
print(func(x))

input_variables=[] input_types={} partial_variables={} template='"""\nUsing the following resume, write a professional and inquisitive cold mail tailored to the provided person\'s information, seeking job opportunities:\n\nResume: Dinesh\tSagar\n+91-8121909150\n\t\nMail\n\t\nGitHub\n\t\nLinkedIn\tProfile\nPassionate\tData\tScientist\twith\ta\tdrive\tto\ttackle\treal-world\tchallenges\tthrough\tdata\tanalysis\tand\tmachine\tlearning.\nCommitted\tto\textracting\tvalue\tfrom\tcomplex\tdata\tand\tdelivering\tactionable\tinsights\tto\tinform\tbusiness\tdecisions\t\nand\ndrive\tgrowth.\tEager\tto\tcontinuously\tlearn\tand\timprove\tto\tdeliver\timpactful\tsolutions.\nExperience\nCapgemini\nConsultant\nFEB\t2025-\tPRESENT\nAmazon\tDevelopment\tCentre\tIndia\nQuality\tSpecialist\nJAN\t2020\t-\tDEC\t2024\nImplemented\tan\toutlier\tdetection\tprogram\tto\nenhance\taddress\tdata\tquality,\timproving\tlogistics\nperformance\tfor\tAmazon\tLogistics.\nAnalyzed\tlarge-scale\tdatasets\tusing\tstatisti